In [1]:
# import
import pandas as pd
import numpy as np
import selenium
import sqlalchemy
from urllib import parse
import requests
from bs4 import BeautifulSoup as BS
import os
import json
from tqdm import tqdm
import pickle
from geopy.geocoders import Nominatim
import subprocess
import time
import lxml

# custom python lib
import src.naver_geocoding_api as nga
import src.evchargestations as evcs
import src.connect_to_db as cdb


In [2]:
with open("./results/EV_CS_DATA.pkl", "rb") as f:
    cs_info_df = pickle.load(f)

In [3]:
cs_info_df.loc[2969]

metro                                                    충청남도
city                                                      보령시
stnPlace                                               농업기술센터
stnAddr     충청남도 null null null null null null null null n...
rapidCnt                                                    1
slowCnt                                                     0
carType                                           SM3 Z.E,테슬라
Name: 2969, dtype: object

In [4]:
data = evcs.extract_charge_stations()

/home/leshen/repository/EVDA/src/evchargestations.py:59: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 59 of the file /home/leshen/repository/EVDA/src/evchargestations.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BS(req.text)
100%|██████████| 16/16 [00:00<00:00, 1809.21it/s]


In [2]:
with open("./data/gas_station_final.pkl", "rb") as f:
    gas_station_df = pd.read_pickle(f)

In [5]:
gas_station_df.shape

(10766, 6)

In [6]:
testdf = gas_station_df.head(5).copy()

In [9]:
testdf.head(5)

,sido,sigungu,station_name,address,company,phone_number,coordinates
0,서울특별시,강남구,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),SK에너지,02-445-6870,"[127.1187331, 37.4672785]"
1,서울특별시,강남구,HD현대오일뱅크㈜직영 산성셀프주유소,서울 강남구 헌릉로 730,HD현대오일뱅크,02-2226-4963,"[127.1190231, 37.4665975]"
2,서울특별시,강남구,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),SK에너지,02-445-5841,"[127.1040159, 37.4835419]"
3,서울특별시,강남구,방죽주유소,서울 강남구 밤고개로 215 (율현동),GS칼텍스,02-459-3434,"[127.1068180, 37.4749811]"
4,서울특별시,강남구,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),SK에너지,02-3462-5100,"[127.0395832, 37.4858704]"


In [8]:
tqdm.pandas()
testdf['coordinates'] = testdf['address'].progress_apply(lambda x: nga.geocoding(x))

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:02<00:00,  1.89it/s]


In [3]:
tqdm.pandas()
gas_station_df['coordinates'] = gas_station_df['address'].progress_apply(lambda x: nga.geocoding(x))
with open("./data/gas_station_final_with_coor.pkl", "wb") as f:
    pickle.dump(gas_station_df, f)

100%|██████████| 10766/10766 [2:04:49<00:00,  1.44it/s]  


In [10]:
with open("./data/gas_station_final_with_coor.pkl", "rb") as f:
    gas_data = pd.read_pickle(f)

In [14]:
gas_data.shape

(10766, 7)

In [13]:
gas_data[gas_data['coordinates'].apply(lambda x:x[0]==0 and x[1]==0)].shape

(43, 7)

In [230]:
testdf[testdf['coordinates'].apply(lambda x:x[0]==0 and x[1]==0)]

,sido,sigungu,station_name,address,company,phone_number,coordinates


In [1]:
pip install pyshorteners

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pyshorteners: filename=pyshorteners-1.0.1-py3-none-any.whl size=17498 sha256=0296da005901270d252de1cbe55702cf4363c7e4ceb7b85376402029afc5a7f0
  Stored in directory: c:\users\playdata\appdata\local\pip\cache\wheels\d5\ff\b3\16d8906f92ab0b042f76a4696fcea11d291d79a2b3a3de1e78
Successfully built pyshorteners
Note: you may need to restart the kernel to use updated packages.


In [6]:
url = """
https://stat.molit.go.kr/portal/cate/statView.do?hRsId=58&hFormId=5498&hSelectId=5498&hPoint=00&hAppr=1&hDivEng=&oFileName=&rFileName=&midpath=&sFormId=5498&sStart=201101&sEnd=202403&sStyleNum=2&settingRadio=csv
"""

In [7]:
import pyshorteners
print(pyshorteners.Shortener().tinyurl.short(url.strip()))

https://tinyurl.com/23ha88m4
